# Optimizing Dry Bean Sorting into Classes using Classification with K-nearest neighbors

## Introduction:

Dry beans, scientifically known as Phaseolus vulgaris L., come in different varieties and are an important crop produced globally (Beebe et al., 2013). Cultivating mixed bean species rather than a single variety is common in Turkish agriculture, as well as in other Asian countries. This results in economic losses, as it is more difficult to process and sell mixed bean types. Turkey has several varieties of dry beans, including Barbunya, Battal, Bombay, Calı, Dermason, Horoz, Tombul, Selanik, and Seker. However, the market value of dry beans depends on the specific type of bean, and some beans are sold for higher prices than others.

The cultivation of the seven different beans together makes sorting them a crucial task (Koklu & Ozkan, 2020). Rather than selling them as a mixed batch, farmers can make more profit by sorting them by bean type. Manual classification of bean seeds can be error-prone and time-consuming, especially when dealing with high production volumes. Manual sorting is also expsensive and inconsistent concerning accuracy, which has led to growing interest in automatic sorting systems. Computer algorithms can analyze images of seeds and classify them based on features like size and shape, making sorting efficient and less labor-intensive than manual sorting.  A seed classification system would assist farmers in obtaining uniform seed varieties from crop production while reducing the time and error of manual sorting.

In this project, we are interested in seed classification within the context of our [dataset](https://archive.ics.uci.edu/ml/datasets/dry+bean+dataset), which identifies the seven different types of dry beans cultivated in Turkey based on the market situation and their physical features according to the Turkish Standards Institution. This research uses images of seven varieties of dry beans generated by a computer vision system, examining 16 features of the grains (Koklu & Ozkan, 2020). 

Our aim in this project is to create a k-NN classifier that can classify dry bean seeds uniformly and accurately, using the dataset from this study.

**Predictive question**: Can we accurately predict bean type (Seker, Barbunya, Bombay, Cali, Dermosan, Horoz, and Sira) using the physical features of the dry beans? 

Figure 1: Image of the Seven Types of Dry Beans

![](https://ars.els-cdn.com/content/image/1-s2.0-S0168169919311573-gr3.jpg)

source: https://www.sciencedirect.com/science/article/pii/S0168169919311573

### Variables of Interest:

| Variable Name  | Description                                 |
|--------|---------------------------------------------|
| Area (A)     | The area of a bean zone and the number of pixels within its boundaries.            |
|Perimeter (P)    | Bean circumference is defined as the length of its border.         |
| Major axis length (L)    | The distance between the ends of the longest line that can be drawn from a bean     |
|  Minor axis length (l)    | The longest line that can be drawn from the bean while standing perpendicular to the main axis |
| Aspect ratio (K)     | Defines the relationship between L and l           |
|Eccentricity (Ec)    | Eccentricity of the ellipse having the same moments as the region        |
| Convex area (C)    | Number of pixels in the smallest convex polygon that can contain the area of a bean seed      |
|  Equivalent diameter (Ed)   | The diameter of a circle having the same area as a bean seed area. |
| Extent (Ex)    |The ratio of the pixels in the bounding box to the bean area.            |
|Solidity (S)    | Also known as convexity. The ratio of the pixels in the convex shell to those found in beans          |
| Roundness (R)    |Calculated with the following formula: (4piA)/(P^2)   |
|  Compactness (CO)    |Measures the roundness of an object: Ed/L |
| ShapeFactor1 (SF1)    |Not specified in orginal dataset         |
|ShapeFactor2 (SF2)    |Not specified in orginal dataset         |
| ShapeFactor3 (SF3)   |Not specified in orginal dataset        |
|  ShapeFactor4 (SF4)   |Not specified in orginal dataset  |
| Class   | Bean type (Seker, Barbunya, Bombay, Cali, Dermosan, Horoz and Sira)|


### Methods & Results:

In [5]:
# Download libraries

options(repr.matrix.max.rows = 6)
install.packages("tidyverse")
library(tidyverse)
install.packages("tidymodels")
library(tidymodels)
install.packages("kknn")
library(kknn)

also installing the dependencies ‘systemfonts’, ‘textshaping’, ‘ragg’


Warning message in install.packages("tidyverse"):
“installation of package ‘systemfonts’ had non-zero exit status”
Warning message in install.packages("tidyverse"):
“installation of package ‘textshaping’ had non-zero exit status”
Warning message in install.packages("tidyverse"):
“installation of package ‘ragg’ had non-zero exit status”
Warning message in install.packages("tidyverse"):
“installation of package ‘tidyverse’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



ERROR: Error in library(tidyverse): there is no package called ‘tidyverse’


In [ ]:
# Read dataset

url <- "https://github.com/NaziaNadir/dsci-100-2023-007-group-8/raw/main/Dry_Bean_Dataset.csv"
bean_data <- read_csv(url)|> mutate(Class = as_factor(Class))

# remove missing values if any
bean_data <- filter(bean_data, rowSums(is.na(bean_data)) != ncol(bean_data))

In [ ]:
#Wrangle/clean data 

clean_bean_data <- bean_data |>
mutate(Class= as_factor(Class))

clean_bean_data <- na.omit(clean_bean_data)

glimpse(clean_bean_data)

#### Data set exploration summary


In [ ]:
# Name of each category in Class column
clean_bean_data |>
  pull(Class) |>
  levels()

In [ ]:
# Number and percentage of each category.
 
num_obs <- nrow(clean_bean_data)

bean_number <- clean_bean_data |>
  group_by(Class) |>
  summarize(
    count = n(),
    percentage = n() / num_obs * 100) |>
    arrange(count)

print(bean_number)

The above count table shows that the class DERMASON has the highest number of observations. It occupies about 26.1 % of the total observations in the dataset while seed BOMBAY has the lowest number of observations with about 3.84% of the total observations. The different ratios of observations of each class might affect the classification model. However, the number of observations in each class is not very far off therefore resampling of data is essentially not required. 

In [ ]:
set.seed(25)
#Create the train/ test split
bean_split <- initial_split(clean_bean_data, prop = 0.50, strata = Class)
bean_train <- training(bean_split)
bean_test <- testing(bean_split)

In [ ]:
glimpse(bean_train)

In [ ]:
glimpse(bean_test)


We can observe from glimpse in the code above that the training set contains 6804 observations, while the test set contains 6807 observations. This corresponds to a train / test split of 50% / 50%, as desired.

In [ ]:
# Predictor selection
options(repr.plot.height = 9, repr.plot.width = 15)
density_plot <- bean_train |>
    pivot_longer(cols = -Class, names_to = 'category', values_to = 'value')  |> 
    ggplot(aes(value, fill = Class)) +
        geom_density(alpha = 0.4, colour = 'white') +
        facet_wrap(facets = vars(category), scales = 'free') +
        theme_minimal() +
        labs(x="Distribution of Bean Type across Variables", y="Count", fill="Bean Type")+
        ggtitle("Figure 2: Using Density Plots To Perform Variable Section")+
        theme(text = element_text(size = 20)) +
 theme(axis.text.x = element_text(angle = 45, hjust =1)) +
  scale_x_log10(labels = label_comma())
density_plot

To choose our predictors:

First, we removed variables we don't understand because the original dataset did not explain them. These include shape factors 1, 2, 3, and 4.

Next, we removed variables that convey similar things making them redundant. When it comes to using them as predictors, this will cause our model to place a greater emphasis on those predictors since they are accounted for twice. 
The aspect ratio is the relationship between the major and minor axis of an object. So, we removed the major axis and minor axis. The Area is similar to the Perimeter. So, we removed the area.

Lastly, we utilized histograms to visualize the distribution overlap of different variables. If the distributions between the graphs greatly overlap then it might not be a good predictor because if an unknown observation was used, it'd be within a cluster of many different bean types. If one distribution was skewed to the right and one distribution was skewed to the left (i.e. very little overlap), that could be an indication of a good predictor. Referring to figure 2, distribution of the solidity graph shows notable overlap between every class of bean; thus, this variable was removed.

Therefore, we are left with the following variables to be used in our classifier: **AspectRation, Compactness, ConvexArea, Eccentricity, EquivDiameter, Extent, Perimeter, and Roundness.**

In [ ]:
#Update data set using only the chosen predictors
bean_train <- bean_train |>
select("AspectRation", "Compactness", "ConvexArea", "Eccentricity",
       "EquivDiameter", "Extent", "Perimeter", "roundness", "Class")

bean_test <- bean_test |>
select("AspectRation", "Compactness", "ConvexArea", "Eccentricity",
       "EquivDiameter", "Extent", "Perimeter", "roundness", "Class")

# Number each category in the test data for accuracy evaluation.

test_bean_number <- bean_test |>
  group_by(Class) |>
  summarize(
    count = n()) |>
    arrange(count)

print(test_bean_number)

## **Variable means and standard deviations**

In [ ]:
bean_mean <- bean_train|>
group_by(Class)|> 
summarise(mean_Perimeter = mean(Perimeter), 
                  mean_AspectRation = mean(AspectRation),
                  mean_Eccentricity = mean(Eccentricity),
                  mean_EquivDiameter = mean(EquivDiameter),
                  mean_roundness = mean(roundness),
                  mean_Compactness = mean(Compactness),
                  mean_Extent = mean(Extent), 
                  mean_ConvexArea = mean(ConvexArea))
print(bean_mean)




​​The table above shows the average of each variable by class. For Perimeter and EquivDiameter, SEKER and SIRA have similar means while BOMBAY has the highest mean. For roundness, and compactness, HOROZ, CALI, and BOMBAY have similar means while SEKER has the highest mean. For AspectRatio and Eccentricity, SEKER, DERMASON, and BARBUNYA have similar means while HOROZ has the highest mean.The above table summarizes the mean of each predictor variable grouped by its Class.
The varibales perimeter and ConvexArea have  highest mean value for BOMBAY and lowest for DERMASON. 
The mean of AspectRation and Eccentricity is highest for HOROZ while it is lowest for SEKER. 
The mean EquivDiameter is highest for BOMBAY  and it is lowest for  DERMASON .
The variables roundness, compactness and extent have highest mean values for seed SEKER and and lowest for HOROZ . 





In [ ]:
# Table of standard deviation for each predictor by Class
bean_sd <- bean_train|>
group_by(Class)|>
summarise(sd_Perimeter = sd(Perimeter), 
          sd_AspectRation = sd(AspectRation),
          sd_Eccentricity = sd(Eccentricity),
          sd_EquivDiameter = sd(EquivDiameter),
          sd_roundness = sd(roundness),
          sd_Compactness = sd(Compactness),
          sd_Extent = sd(Extent),
          sd_ConvexArea = sd(ConvexArea))
          
print(bean_sd) 

The above table shows the spread of values for each predictor variable grouped by its Class. The standard deviation shows the spread of variables in a Class. 
The standard deviation for perimeter and ConvexArea is highest for BOMBAY and lowest for SIRA.
The varibales perimeter and ConvexArea have  highest standard deviation value for BOMBAY and lowest for DERMASON. 
The standard deviation of AspectRation and Eccentricity is highest for HOROZ while it is lowest for SEKER. 
The standard deviation EquivDiameter is highest for BOMBAY  and it is lowest for  DERMASON .
The variables roundness, compactness and extent have highest standard deviation values for seed SEKER and and lowest for HOROZ . 


### K Value Selection

In [ ]:
# Five fold Cross validation
bean_vfold <- vfold_cv(bean_train, v = 5, strata = Class)

In [ ]:
# Model specification
knn_spec <- nearest_neighbor(weight_func = "rectangular", 
                             neighbors = tune()) |>
                             set_engine("kknn") |>
                             set_mode("classification")
knn_spec

In [ ]:
# Generate the recipe and standarize the variables
bean_recipe <- recipe(Class ~ ., data = bean_train) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())
bean_recipe

In [ ]:
# Generate a tibble to test different K values
k_vals <- tibble(neighbors = seq(from = 1, to = 15, by = 1))
k_vals

In [ ]:
# Feed the recipe and model specification into a workflow
knn_results <- workflow() |>
  add_recipe(bean_recipe) |>
  add_model(knn_spec) |>
  tune_grid(resamples = bean_vfold, grid = k_vals) |>
  collect_metrics() 
# Output the accuracy values
accuracies <- knn_results |>
  filter(.metric == "accuracy")

In [ ]:
# Find the maximum accuracy
accuracies_max <- accuracies |>
arrange(mean)|>
tail(1)
accuracies_max

In [ ]:
# Generate the graph to choose optimal K value
options(repr.plot.width = 12, repr.plot.height = 7)

ky <- accuracies |>
    filter(neighbors == 13) |>
    select(mean) |>
    pull()

cross_val_plot <- accuracies |> 
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    geom_vline(xintercept = 13, colour = "red") +
    geom_point(aes(x = 13, y = ky), size = 4, colour = "blue") +
    labs(x = "\nNeighbours",
         y = "Accuracy Estimate\n",
         title = "Estimated Accuracies of k Neighbours") +
    scale_x_continuous(limits = c(1, 16),
                       breaks = seq(1, 16),
                       minor_breaks = seq(1, 16, 1)) +
    theme(text = element_text(size = 20),
          plot.caption = element_text(size = 20, hjust = 0))+
ggtitle("Figure 3a: Selecting the Best K Value")

cross_val_plot

Based on the findings presented in Figure 3a, it can be concluded that a value of k=13 provides the highest estimated accuracy. Moreover, the graph indicates that the estimated accuracy remains relatively stable even if we slightly increase or decrease k around the value of 13. Furthermore, selecting k=13 does not result in a computationally expensive training process. Therefore, considering all these factors, we would choose k=13 as the optimal value for the classifier.

In [ ]:
# Make a new model specification for the best parameter value 

In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 13) |>
  set_engine("kknn") |>
  set_mode("classification")

knn_fit <- workflow() |>
  add_recipe(bean_recipe) |>
  add_model(knn_spec) |>
  fit(data = bean_train)
knn_fit

In [ ]:
# Evaluate the estimated accuracy of the classifier 

In [ ]:
bean_predictions <- predict(knn_fit, bean_test) |>
                    bind_cols(bean_test)
bean_predictions

In [ ]:
levels(bean_predictions$.pred_class) <- levels(bean_predictions$Class)
bean_predictions |>
metrics(truth = Class, estimate =.pred_class)|>
filter(.metric == "accuracy")

# Summary Table

In [ ]:
# Generate confusion matrix
confusion <- bean_predictions |>
             conf_mat(truth = Class, estimate = .pred_class)
confusion

In [ ]:
# Bar graph showing true positive, true negative, false positive, false negative and accuracies for each
# class based on the confusion matrix
confusion_values = c(948,5725,72,62,round((((948+5725)/(946+5725+72+62))*100),2),
                     550, 6126,34,97,  round((((550+6126)/(550+6126+34+97))*100),2), 
                     256, 6551,0,0, round((((256+6551)/(256+6551+0+0))*100), 2),
                     782, 5888, 97,40,round((((782+5888)/(785+5888+97+40))*100),2), 
                     902,5820, 23, 62,  round((((902+5820)/(902+5820+23+62))*100),2),
                     1175, 5245, 226,157,  round((((1175+5245)/(1175+5245+226+157))*100),2),
                     1600, 5061,142, 176, round((((1600+5061)/(1600+5061+142+176))*100),2))
    
confusion_values
class_name <- c(rep("SEKER",5), rep("BARBUNYA",5), rep("BOMBAY",5),rep("CALI",5),rep("HOROZ",5),rep("SIRA",5),rep( "DERMASON",5))
conf_outcomes <- c(rep(c("True Positive", "True Negative", "False Positive", "False Negative", "Percentage Accuracy"), 7))

conf_tibble <- tibble(class_name, conf_outcomes, confusion_values)   
conf_tibble 

bean_bar <- ggplot(conf_tibble, aes(x = conf_outcomes, y = confusion_values, fill= conf_outcomes)) +
  geom_bar(stat = "identity", positio = position_dodge()) +
  labs(x="Bean Class", y = "Count", fill = "Confusion Matrix Outcomes") +
  ggtitle("Figure 4: Bar Plot Visualization of Confusion Matrix Results") +
  facet_wrap(~class_name)+
  geom_text(aes(label = confusion_values), position = position_dodge(width = 0.2), vjust = -0.25, size=3)+
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))+
  theme(text = element_text(size=12))
  

bean_bar 
                                                                                                                                                                                                           

In [ ]:
## description of bars
sensitivity and accuracy formula



In [ ]:
# Heat map based on the confusion matrix
value =c(948,  7,  0,    2,     0,   19 ,      44,
      4,   550,      0,   21,     3,    5,  0,
      0,    0,    256,    0,     0,    0,    0,
      0,    74,      0,  782,    20,    3,    0,
      0,     3,      0,    8,   902,   10,    2,
      43,    13,      0,    9,    33, 1175,   129,
      16,     0,      0,   0,     6,  120,    1600)
predicted <- c(rep(c("SEKER", "BARBUNYA", "BOMBAY", "CALI", "HOROZ", "SIRA", "DERMASON"), 7))
actual <- c(rep("SEKER",7), rep("BARBUNYA",7), rep("BOMBAY",7),rep("CALI",7),rep("HOROZ",7),rep("SIRA",7),rep( "DERMASON",7))

conf_matriz <- tibble(actual, predicted, value)


# Heatmap for correlation of variables
heat_map <- ggplot(conf_matriz, aes(x = predicted, y= actual , fill = value)) +
  geom_tile(color = "white",
            lwd = 1.5,
            linetype = 1)+
    xlab("Predicted Bean Class")+
    ylab("Actual Bean Class")+
  geom_text(aes(label = value), color = "black", size = 4)+
  scale_fill_gradientn(colors = hcl.colors(30, "RdYlGn")) +
 guides(fill = guide_colourbar(title = "Count Values"))+
  ggtitle("Figure 5: Heatmap Visualization of Confusion Matrix Results")+
    theme(text=element_text(size=15))
heat_map 


Prices for different classes of beans from online Turkish stores

            SEKER: $10/kg. (Marché Istanbul, (n.d.))

            BARBUNYA: $25.73/kg  (Grand Turkish Bazaar, (n.d.))

            SIRA: $6.50/kg.  (Marketim,(n.d.))

            HOROZ: 6$/kg.  (Turkish Grocery, (n.d.))

            BOMBAY: 10.81$/kg. (Grand Turkish Bazaar, (n.d.))

            CALI: $10.79/kg (O.M. Foods, (n.d.))

            DERMASON: $10.30/kg. (Grand Turkish Bazaar, (n.d.))

From figure 4 and 5, the classifier accurately predicts all the beans for the class BOMBAY leading to a 100% accuracy for this class. It classifies 550 BARBUNYA correctly out of 647 with an accuracy of 98.08%. Moreover, 782 CALI beans were correctly classified out of 822 leading to an accuracy of 97.94% for this class. 902 HOROZ beans out of 964 were correctly classified with an accuracy of  98.75% for this class. 
948 SEKER beans out of 1010 were correctly classified with an accuracy of 98.06% . 1175 SIRA out of 1332 were correctly classified out of 822 leading to an accuracy of 94.37% and 1600 DERMASON out of 1776 were correctly classified with an accuracy of 95.44%. 

From Figures 4 and 5, the five classes of beans i.e SEKER, BARBUNYA, BOMBAY, CALI, and HOROZ were correctly classified about 98% of the time. In this regard, the classifier is performing well especially because CALI, BOMBAY, and BARBUNYA are some of the highest-priced bean varieties.  The accuracy for SIRA and DERMASON is 94.58% and 95.53% respectively. They also have high false positive values. The low accuracy for DERMASON is especially a concern since it is among the highly valued and high-priced beans. 



### Discussion:

summarize what you found 


As seen in the above data analysis, the KNN classifier has an accuracy of about 91% in classifying seven different classes of seed which are 'SEKER', 'BARBUNYA', 'BOMBAY', 'CALI', 'HOROZ', 'SIRA', and 'DERMASON'. The confusion matrix and heatmap represent the summarisation of the predicted results (Figure 4 and 5), which is a significant tool for evaluating the accuracy of our classifier model.

The above heatmap, Figure 5, gives a visual representation of the data analysis and accuracy of the model. The overall accuarcy of the model is sufficicent for our purposes as it predicts about 91% correct observations. Notably, there were some seed types that were misclassified. The diagonal of the heatmap depicts the correct predictions made by the model. We can draw a few conclusions from the heatmap, which represents misclassified seeds (lower counts) with red colour, while it depicts high misclassfication with orange. By observing the heatmap we find that about 120 DERMASON seeds were identified SIRA (as it has an orange colour) and about 45 DERMASON seeds were identified SEKER (light orange). Also, about 122 SIRA seeds were identified DERMASON and therefore, having orange colour for depiction.This suggests that there might some physical similarities between DERMASON and SIRA seeds. In addition to this about 21 CALI seeds were misclassified as BARBUNYA and about 74 BARBUNYA seeds were misclassified as CALI. This suggests that there may be some overlap in the features of these two types of seeds, making it challenging for the classifier to differentiate between them accurately.



discuss whether this is what you expected to find?


Based on the dataset, we did not expect the classification model to achieve a 91% accuracy. The dataset contained 16 different variables, which could be used as predictors in the classification model. The higher number of predictors in a model often corresponds to less accuracy. Other than this the target variable of the dataset which is the CLASS has seven different types of seed. We saw this as a potential reason to decrease the accuracy of the model as the class 'BOMBAY' had about 3.84% observations in the dataset while the class 'DERMASOn' had about 26.1% observations in the dataset. This makes the data close to an imbalanced dataset and might make the model biassed towards one type.Other than this the seven different types of seed types increases the complexity and makes it challenging for the model to predict accurate seed type.

A 91% accuracy classifier can be considered good in some contexts, but it may not be sufficient in others. It ultimately depends on the user's needs and expectations. For our specific classifer, it had difficulty distinguishing between Dermason and Sira beans, which have different market values. Dermason beans are currently around 10 dollars per kg, while Sira beans 6 dollars per kg. This means that misclassifying a significant amount of beans can result in a big difference in profit for farmers. For example, misclassifying 1000 kg of Dermason beans as Sira beans would result in a loss of 4000 dollars. Similarly, misclassifying Barbunya beans as Cali beans could also result in substantial losses for farmers, given the large price difference between them. Barbunya beans are around 27 dollars per kg, while Cali beans (also known as Cannellini beans) are around 10 dollars per kg. Misclassifying 1000 kg of Barbunya beans as Cali beans would result in a loss of 17,000 dollars. As such, farmers may have different expectations for their classifier's accuracy. Some may think a 91% accuracy is acceptable, while others may not. If a farmer believes that a 9% error rate is too high and will result in significant losses, they may consider the classifier insufficient. However, if a farmer believes that 91% accuracy is acceptable, they may use the classifier. Therefore, the effectiveness of a classifier is subjective to the user and their need for profit. A 91% accuracy rate may be good enough for some, but not for others.


Price for Dermason: https://www.grandturkishbazaar.com/product/dermason-beans-turkish/ - 10.30 / kg
Price for sira: https://marketim.ca/products/yayla-sira-white-bean-1kg - 6.50 / kg
Price for Barbunya: https://www.grandturkishbazaar.com/product/kidney-beans-barbunya/?aelia_cs_currency=CAD&utm_source=GoogleAds&utm_medium=cpc&utm_campaign=CANADA-Pmax&gclid=CjwKCAjw8-OhBhB5EiwADyoY1ZU4_8L6AzZCdaW6Itqa-meDomp77bQ_E8k0hu622kpwx-8znlNS4BoCdGYQAvD_BwE - 27.37/ kg
Price for Cannellini/Cali beans: https://www.omfoods.com/products/legumes-organic-cannelini-beans?variant=7102367137849&currency=CAD&utm_medium=product_sync&utm_source=google&utm_content=sag_organic&utm_campaign=sag_organic&gclid=CjwKCAjw8-OhBhB5EiwADyoY1Q7sU-hqHXRp5Yt87dL0CnOnIn71czrq9yEiRJqLBiKVJiUliZ4ShRoCGScQAvD_BwE - 10.8/kg


### References

 Beebe, S. E., Rao, I. M., Blair, M. W., & Acosta-Gallegos, J. A. (2013). Phenotyping common beans for adaptation to drought. Frontiers in Physiology, 4, 35-35. 
https://doi.org/10.3389/fphys.2013.00035

 KOKLU , M., &amp; OZKAN , I. A. (2020). Dry Bean Dataset. UCI Machine Learning Repository: Dry Bean Dataset Data set. Retrieved April 13, 2023, from https://archive.ics.uci.edu/ml/datasets/Dry+Bean+Dataset 


 Koklu, M., & Ozkan, I. A. (2020). Multiclass classification of dry beans using computer vision and machine learning techniques. Computers and Electronics in Agriculture, 174, 105507. https://doi.org/10.1016/j.compag.2020.105507
 

 Varankaya, S., & Ceyhan, E. (2012). Problems Encountered in Bean Farming in the Central Anatolia Region and Solution Suggestions. Selçuk Tarım Bilim. Journal. 26, 15–26.

 Image source: https://www.sciencedirect.com/science/article/pii/S0168169919311573

 O.M. Foods. (n.d.). Organic Cannellini Beans. O.M. Foods. Retrieved April 15, 2023, from https://www.omfoods.com/products/legumes-organic-cannelini-beans?variant=7102367137849&currency=CAD&utm_medium=product_sync&utm_source=google&utm_content=sag_organic&utm_campaign=sag_organic&gclid=CjwKCAjw8-OhBhB5EiwADyoY1Q7sU-hqHXRp5Yt87dL0CnOnIn71czrq9yEiRJqLBiKVJiUliZ4ShRoCGScQAvD_BwE


Grand Turkish Bazaar. (n.d.). Kidney Beans (Barbunya). Grand Turkish Bazaar. Retrieved April 15, 2023, from https://www.grandturkishbazaar.com/product/kidney-beans-barbunya/?aelia_cs_currency=CAD&utm_source=GoogleAds&utm_medium=cpc&utm_campaign=CANADA-Pmax&gclid=CjwKCAjw8-OhBhB5EiwADyoY1ZU4_8L6AzZCdaW6Itqa-meDomp77bQ_E8k0hu622kpwx-8znlNS4BoCdGYQAvD_BwE

Marketim. (n.d.). Yayla Sira White Bean 1kg. Marketim. Retrieved April 15, 2023, from https://marketim.ca/products/
yayla-sira-white-bean-1kg

Grand Turkish Bazaar. (n.d.). Dermason Beans Turkish. Grand Turkish Bazaar. Retrieved April 15, 2023, from https://www.grandturkishbazaar.com/product/dermason-beans-turkish/

Marché Istanbul. (n.d.). Reis Sugar Beans. Marché Istanbul. Retrieved April 15, 2023, from https://montreal.marcheistanbul.com/products/reis-sugar-beans

Turkish Grocery. (n.d.). Reis Bombay Broad Beans 1kg. Turkish Grocery. Retrieved April 15, 2023, from https://turkishgrocery.com/reis-bombay-broad-beans-1kg/